In [3]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms

device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
### Hyper paramas:

#dataset
IMG_SIZE = 128
BATCH_SIZE = 32 # maybe it bigger if it cans

#seeds
torch.manual_seed(36)
torch.cuda.manual_seed(36)


In [29]:
transform_data = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

data_ds = torchvision.datasets.OxfordIIITPet(root = "data" , split = "trainval",target_types = "category",download = True,transform=transform_data)


data_dl = DataLoader(data_ds,batch_size=BATCH_SIZE,shuffle=True)

In [6]:
len(data_ds.classes)

37

In [15]:
classes = data_ds.classes
classes_idx = []
for i in range(len(classes)):
  classes_idx.append(i)

classes[1],classes_idx[1]


('American Bulldog', 1)

In [ ]:
linear = nn.Linear(1,100)
x = torch.tensor([1]).to(torch.float)
x = linear(x)
print(x.shape)
print(x)

In [36]:
def weights_init(m):
  classname = m.__class__.__name__
  if classname.find('Conv')!= -1:
    nn.init.normal_(m.weight.data,0.0,0.02)
  elif classname.find('BatchNorm')!= -1:
    nn.init.normal_(m.weight.data, 0.0,0.02)
    nn.init.constant_(m.bias.data,0)

In [101]:
class embedding(nn.Module):
  def __init__(self,out_size):
    super().__init__()
    self.embeding_layer = nn.Sequential(
        nn.Linear(1,32),
        nn.ReLU(inplace=True),
        nn.Linear(32,64),
        nn.SiLU(inplace=True),
        nn.Linear(64,out_size)
    )
  def forward(self,x):
    x = torch.tensor([x]).to(torch.float)
    x = self.embeding_layer(x)
    x = x.unsqueeze(1).unsqueeze(1).unsqueeze(1)
    return x

In [102]:
embed = embedding(100)
x = embed(1)
x.shape

torch.Size([100, 1, 1, 1])

In [103]:
### Discriminator :

def d_block(in_ch,out_ch,kernel=4,stride=2,padding=1):
  return nn.Sequential(
      nn.Conv2d(in_ch,out_ch,kernel,stride,padding,bias=False),
      nn.LeakyReLU(0.2,inplace=True),
      nn.BatchNorm2d(out_ch)
  )
class Discriminator(nn.Module):
  def __init__(self):
    super().__init__()
    self.embed_size = 100
    self.embed = embedding(self.embed_size)
    self.model = nn.Sequential(
        ## bs x 3 x128 x128
        d_block(3,32), # -> 32 x 128 x 128
        d_block(32,64), # -> 64 x 64 x 64
        d_block(64,128), # -> 128 x 32 x 32
        d_block(128,256), # -> 256 x 16 x 16
        d_block(256,512), # -> 512 x 8 x 8
        d_block(512,32), # -> 32 x 4 x4
        nn.Flatten(),
    )
    self.model_2 = nn.Sequential(
        nn.Linear(160,100),
        nn.LeakyReLU(100,1),
        nn.Sigmoid(),
    )
    self.apply(weights_init)

  def forward(self,input,labels):
    x = self.model(input)
    y = self.embed(labels)
    input = torch.cat([x,y],1)
    final_output = self.model_2(input)

    return final_output

In [104]:
discriminator_model = Discriminator().to(device)
discriminator_model

Discriminator(
  (embed): embedding(
    (embeding_layer): Sequential(
      (0): Linear(in_features=1, out_features=32, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=32, out_features=64, bias=True)
      (3): SiLU(inplace=True)
      (4): Linear(in_features=64, out_features=100, bias=True)
    )
  )
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
      (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): Leaky

In [113]:
def g_block(in_ch,out_ch,kernel=4,stride=2,padding=1):
  return nn.Sequential(
      nn.ConvTranspose2d(in_ch,out_ch,kernel,stride,padding,bias=False),
      nn.ReLU(inplace=True),
      nn.BatchNorm2d(out_ch),
  )

class Generator (nn.Module):
  def __init__(self):
    super().__init__()
    self.embed_size = 200
    self.embed = embedding(self.embed_size)
    self.model = nn.Sequential(
        g_block(100+self.embed_size,512,stride=1,padding=0),
        g_block(512,256),
        g_block(256,128),
        g_block(128,64),
        g_block(64,32),
        g_block(32,3),
    )

    self.apply(weights_init)

  def forward(self,input_noise,labels):
    print(input_noise.shape)
    print(labels)
    label_embeddings = self.embed(labels)
    print(label_embeddings.shape)
    input = torch.cat([input_noise,label_embeddings],0)

    return self.model(input)

In [114]:
generator_model = Generator()
generator_model

Generator(
  (embed): embedding(
    (embeding_layer): Sequential(
      (0): Linear(in_features=1, out_features=32, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=32, out_features=64, bias=True)
      (3): SiLU(inplace=True)
      (4): Linear(in_features=64, out_features=200, bias=True)
    )
  )
  (model): Sequential(
    (0): Sequential(
      (0): ConvTranspose2d(300, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm2

In [115]:
def noise(size):
  n = torch.randn(size,100,1,1,device=device)
  return n.to(device)

In [116]:
generator_model(noise(100),1)

torch.Size([100, 100, 1, 1])
1
torch.Size([200, 1, 1, 1])


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 100 but got size 1 for tensor number 1 in the list.

In [ ]:
noise(100).shape

torch.Size([100, 100, 1, 1])

In [ ]:
classes = data_ds.class_to_idx
torch.tensor(np.array(classes["Boxer"])).long()

In [ ]:
x,y = data_ds[1]
generator_model(x,y)

In [ ]:
data_ds.transform

In [ ]:
t = transforms.ToTensor()
t(x)